<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_9_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9- analogous to task 4, but implementing a Bayesian hierarchical regression (with
continuous outcome variable) with sex as binary higher level to predict subject age
(zscored).

In [ ]:
%%capture
!pip install nilearn
!pip install pymc3

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
from sklearn.preprocessing import StandardScaler
import nibabel as nib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc3 as pm
import scipy.stats as stats
import arviz as az
from pymc3 import Bernoulli, Model, HalfCauchy
from pymc3 import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()
atlas_nii = index_img(crad['scorr_mean'], 10) # we will set this to 10 for these exercises in order to have 106 rois, which is close to the 100 rois in Exercises 1-5
masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = StandardScaler().fit_transform(brain_data.ext_vars.age[:, None])[:, 0] # gives subject age on standard units after z-scoring

In [ ]:
# We delete the last 6 columns from "input_variables" so that we have the same number of input_features as subjects (100, 100)
input_variables = input_variables[:, :-6]

In [ ]:
# We extract the sex information from the dataset and binarize it
male_female = brain_data.ext_vars.mf
male_female = np.where(np.isin(male_female, 'M'), 0, 1)

In [ ]:
# We already know that ther are 64 females and 36 males, so let's define the model
with pm.Model() as model:

    # We define the Gaussian hyperprior for the mu-parameter of the lower-level region slopes
    hyperprior = pm.Normal('hyperprior', mu=0, sd=1, shape=100)

    # We define sex as a categorical distribution with 2 categories (male, female) with unequal probabilities
    sex = pm.Categorical('sex', p=np.array([0.36, 0.64]), observed=male_female)
   
    # We define the intercept term as a normal distribution with priors (mu=0, sd=1)
    intercept = pm.Normal('intercept', mu=0, sd=1)
    
    # We define the bottom-level region slopes as a normal distribution with priors mu=hyperprior and  sd=1
    beta_coefficients = pm.Normal('beta_coefficients', mu=hyperprior, sd=1, shape=100)
       
    # We calculate the linear model using input_variables, sex, and the model parameters
    linear_model = intercept + input_variables @ beta_coefficients + sex
    
    # We define the likelihood as another normal distribution with mean equal to linear model and sd=1
    likelihood = pm.Normal('likelihood', mu=linear_model, sd=1, observed=output_variable)
    
    # We sample from the posterior distribution using 500 MCMC draws
    trace = pm.sample(500, return_inferencedata=True)

# We summarize the results of the MCMC sampling by calling PyMC3 instead of arviz this time
stats = pm.summary(trace, hdi_prob=0.9)


In [ ]:
# Let's take a look at the results
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=201)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
hyperprior[0],0.109,0.768,-1.128,1.385,0.021,0.027,1352.0,578.0,1.0
hyperprior[1],-0.086,0.769,-1.492,1.069,0.020,0.026,1442.0,715.0,1.0
hyperprior[2],-0.025,0.718,-1.133,1.098,0.020,0.022,1299.0,811.0,1.0
hyperprior[3],-0.045,0.805,-1.362,1.211,0.019,0.030,1798.0,658.0,1.0
hyperprior[4],0.106,0.803,-1.168,1.414,0.023,0.021,1179.0,847.0,1.0
...,...,...,...,...,...,...,...,...,...
beta_coefficients[95],0.299,0.480,-0.414,1.113,0.020,0.014,568.0,719.0,1.0
beta_coefficients[96],-0.016,0.629,-0.977,1.041,0.026,0.018,580.0,738.0,1.0
beta_coefficients[97],0.275,0.548,-0.552,1.215,0.018,0.015,879.0,717.0,1.0
beta_coefficients[98],-0.370,0.690,-1.528,0.712,0.027,0.019,671.0,624.0,1.0
